In [1]:
from sklearn import metrics
import tensorflow as tf
import csv
from threading import Thread
from  read_proces_data import *

In [2]:
sess = tf.InteractiveSession()

In [3]:
# create lstm cell in tensorflow
hidden_units = 128
lstm_cell = tf.contrib.rnn.BasicLSTMCell(num_units =\
        hidden_units,state_is_tuple=True)

batch_size = 20
max_seq = 3260
feature_vec_len = 57
x = tf.placeholder(tf.float32, shape = [batch_size,max_seq,feature_vec_len])
s_len = tf.placeholder(tf.int32,shape=[batch_size])
# run lstm over different sequence length
output_, state = tf.nn.dynamic_rnn(\
        lstm_cell,\
        x,\
#        initial_state = i_state,\
        sequence_length=s_len,\
        dtype=tf.float32)
# take all batch, last output of each batch and full output vector
def take_subarray(array , index):
    return array[range(0,batch_size),index-1]
output = tf.py_func(take_subarray,[output_,s_len],tf.float32)
#print output
# define final output value
target_value = tf.placeholder(tf.float32, shape=[batch_size])
# define weights and bias from output of lstm cell to network final output
rv1 = tf.truncated_normal([lstm_cell.output_size, 2 * lstm_cell.output_size],\
        stddev=0.1,dtype=tf.float32)
Wo1 = tf.Variable(rv1,dtype=tf.float32)

rv2 = tf.truncated_normal([2 * lstm_cell.output_size, 1],\
        stddev=0.1,dtype=tf.float32)
Wo2 = tf.Variable(rv2,dtype=tf.float32)

rb1 = tf.constant(0.1,shape=[2 * lstm_cell.output_size],dtype=tf.float32)
b1 = tf.Variable(rb1,dtype=tf.float32)

rb2 = tf.constant(0.1,shape=[1],dtype=tf.float32)
b2 = tf.Variable(rb2,dtype=tf.float32)
# initialize variables
# calculate final output

s1  = tf.matmul(output, Wo1) + b1
o1 = tf.nn.relu(s1)
logits  = tf.matmul(o1, Wo2) + b2
observed = tf.sigmoid(logits)
#observed = tf.nn.relu(logits)

# calculate cost as calculated in logistic classification.
logistic_cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(\
        labels = target_value, logits = tf.reshape(logits,[batch_size])))
#cost = tf.reduce_mean(-1.0 * target_value * tf.log(observed) - \
#        (1 - target_value) * tf.log(1 - observed))
#cost = tf.reduce_mean(-1 * target_value * tf.log(observed))
#cost = tf.reduce_mean(tf.square(target_value - observed))
# optimize the cost
train_step = tf.train.AdamOptimizer(1e-4).minimize(logistic_cost)
def threshold_fn(array):
    return array >= 0.50
obser = tf.py_func(threshold_fn,[observed], tf.bool)
observe = tf.cast(obser,tf.float32)

sess.run(tf.global_variables_initializer())

In [4]:
# find max no of times to roll back
#max_seq_len = find_max_seq_len()
#print max_seq_len
max_seq_len = 3259

In [34]:
inde_featuress = []
sseq_len = []
target_valuess = []
def data_processing(file_name):
    # get features from current indexed file
    features = get_train_data(file_name)
    # get target values corresponding to this file
    target_valuess.append(get_target_values(file_name))
    # training features is np array with max size and padding
    training_features, s_l = prepare_data(features, max_seq_len)
    sseq_len.append(s_l)
    # scale features to have 0 mean and 1 variance
    scaled_features = feature_scaling(training_features, batch_size,\
            s_l)
    # check if there is any NaN or Inf entries in data
    #isnan = np.any(np.isnan(scaled_features))
    #if True == isnan:
    #    print "nan entry found for file %s"%file_
    #isinf = np.any(np.isinf(scaled_features))
    #if True == isinf:
    #    print "inf entry found for file %s"%file_
    # find independent components from given features
    inde_featuress.append(independent_components(scaled_features, batch_size,\
            s_l))

In [38]:
def train_eval(file_, inde_features, target_values, seq_len):
    for i in range(0,500):
        tr_fe, sl , tv =\
        sample_data(inde_features,target_values,seq_len,batch_size)

        sess.run(train_step, feed_dict = {target_value:tv,\
                x:tr_fe, s_len:sl})

        if i%100 == 0:
            print "iteration %d"%i
            print "cost %f"%sess.run(logistic_cost, feed_dict = {target_value:tv,\
                x:tr_fe, s_len:sl})

            y = sess.run(observe, feed_dict = {x:tr_fe, s_len:sl})
#        print tv
#        print y

            f1s = metrics.f1_score(tv,y)
            print "f1-score is %f"%f1s
            fpr, tpr, thresholds = metrics.roc_curve(tv, y, pos_label=1)
            auc = metrics.auc(fpr,tpr)
            print "area under curve is %f"%auc
            sk_accuracy = metrics.accuracy_score(tv,y)
            print "accuracy %f"%sk_accuracy

            # evaluate over whole file
            y = np.zeros(0)
            if "Sess05" in file_:
#                print file_
#                print inde_features.shape
                for index in range(0,100,batch_size):
#                    print inde_features[index:index+batch_size].shape                    
                    #print inde_features.shape
                    yfile = sess.run(observe, feed_dict = {\
                        x:inde_features[index:index+batch_size], \
                        s_len:seq_len[index:index+batch_size]})
                    y = np.append(y,yfile)
            else:
                for index in range(0,60,batch_size):
                    yfile = sess.run(observe, feed_dict = {\
                        x:inde_features[index:index+batch_size], \
                        s_len:seq_len[index:index+batch_size]})
                    y = np.append(y,yfile)

            f1score = metrics.f1_score(target_values,y)
            print "f1-score of file is %f"%f1score
            fprr, tprr, thresholds = metrics.roc_curve(target_values, y, pos_label=1)
            aucc = metrics.auc(fprr,tprr)
            print "area under curve of file is %f"%aucc
            sk_acc = metrics.accuracy_score(target_values,y)
            print "accuracy of file is %f"%sk_acc
            print " "


In [ ]:
# number of times to iterate over whole training dataset
check_1_time = 1
iter_over_files =  10
for itera in range(0, iter_over_files):
    for file_ in file_dir:
        data_thread = Thread(target=data_processing,args=(file_,))
        data_thread.start()
        data_thread.join()
#        print "reading data file %s"%file_
        
        thread_inde_features = []
        thread_seq_len = []
        thread_target_values = []
        
        thread_inde_features = inde_featuress
        thread_seq_len = sseq_len
        thread_target_values = target_valuess
        
        inde_featuress = []
        sseq_len = []
        target_valuess = []
        
        if 1!=check_1_time:
            tr_ev_thread.join()
#        print thread_inde_features[0].shape
        tr_ev_thread = Thread(target = train_eval, args = \
            (file_, thread_inde_features[0],thread_target_values[0],thread_seq_len[0]))
        tr_ev_thread.start()
        check_1_time = 0

iteration 0
cost 0.609155
f1-score is 0.714286
area under curve is 0.600000
accuracy 0.600000
f1-score of file is 0.876190
area under curve of file is 0.510000
accuracy of file is 0.783333
 
iteration 100
cost 0.651185
f1-score is 0.307692
area under curve is 0.550000
accuracy 0.550000
f1-score of file is 0.295082
area under curve of file is 0.490000
accuracy of file is 0.283333
 
iteration 200
cost 0.621218
f1-score is 0.695652
area under curve is 0.650000
accuracy 0.650000
f1-score of file is 0.882353
area under curve of file is 0.600000
accuracy of file is 0.800000
 
iteration 300
cost 0.659421
f1-score is 0.642857
area under curve is 0.500000
accuracy 0.500000
f1-score of file is 0.893204
area under curve of file is 0.610000
accuracy of file is 0.816667
 
iteration 400
cost 0.607029
f1-score is 0.800000
area under curve is 0.750000
accuracy 0.750000
f1-score of file is 0.921569
area under curve of file is 0.720000
accuracy of file is 0.866667
 
iteration 0
cost 0.780653
f1-score is

iteration 300
cost 0.584668
f1-score is 0.720000
area under curve is 0.650000
accuracy 0.650000
f1-score of file is 0.907407
area under curve of file is 0.562500
accuracy of file is 0.833333
 
iteration 400
cost 0.656740
f1-score is 0.692308
area under curve is 0.600000
accuracy 0.600000
f1-score of file is 0.907407
area under curve of file is 0.562500
accuracy of file is 0.833333
 
iteration 0
cost 0.696433
f1-score is 0.666667
area under curve is 0.500000
accuracy 0.500000
f1-score of file is 0.876404
area under curve of file is 0.438202
accuracy of file is 0.780000
 
iteration 100
cost 0.706200
f1-score is 0.666667
area under curve is 0.550000
accuracy 0.550000
f1-score of file is 0.881356
area under curve of file is 0.483657
accuracy of file is 0.790000
 
iteration 200
cost 0.710721
f1-score is 0.642857
area under curve is 0.500000
accuracy 0.500000
f1-score of file is 0.881356
area under curve of file is 0.483657
accuracy of file is 0.790000
 
iteration 300
cost 0.699884
f1-score 

iteration 100
cost 0.766609
f1-score is 0.285714
area under curve is 0.500000
accuracy 0.500000
f1-score of file is 0.296296
area under curve of file is 0.484127
accuracy of file is 0.366667
 
iteration 200
cost 0.692143
f1-score is 0.307692
area under curve is 0.550000
accuracy 0.550000
f1-score of file is 0.296296
area under curve of file is 0.484127
accuracy of file is 0.366667
 
iteration 300
cost 0.713052
f1-score is 0.307692
area under curve is 0.550000
accuracy 0.550000
f1-score of file is 0.301887
area under curve of file is 0.511905
accuracy of file is 0.383333
 
iteration 400
cost 0.700260
f1-score is 0.400000
area under curve is 0.400000
accuracy 0.400000
f1-score of file is 0.301887
area under curve of file is 0.511905
accuracy of file is 0.383333
 
iteration 0
cost 0.658983
f1-score is 0.181818
area under curve is 0.550000
accuracy 0.550000
f1-score of file is 0.340426
area under curve of file is 0.602564
accuracy of file is 0.483333
 
iteration 100
cost 0.589413
f1-score 

iteration 400
cost 0.631421
f1-score is 0.720000
area under curve is 0.650000
accuracy 0.650000
f1-score of file is 0.795455
area under curve of file is 0.619617
accuracy of file is 0.700000
 
iteration 0
cost 0.672671
f1-score is 0.640000
area under curve is 0.550000
accuracy 0.550000
f1-score of file is 0.600000
area under curve of file is 0.466667
accuracy of file is 0.466667
 
iteration 100
cost 0.734241
f1-score is 0.461538
area under curve is 0.300000
accuracy 0.300000
f1-score of file is 0.600000
area under curve of file is 0.466667
accuracy of file is 0.466667
 
iteration 200
cost 0.664395
f1-score is 0.560000
area under curve is 0.450000
accuracy 0.450000
f1-score of file is 0.600000
area under curve of file is 0.466667
accuracy of file is 0.466667
 
iteration 300
cost 0.655577
f1-score is 0.740741
area under curve is 0.650000
accuracy 0.650000
f1-score of file is 0.625000
area under curve of file is 0.500000
accuracy of file is 0.500000
 
iteration 400
cost 0.675460
f1-score 

iteration 200
cost 0.715236
f1-score is 0.560000
area under curve is 0.450000
accuracy 0.450000
f1-score of file is 0.671533
area under curve of file is 0.550000
accuracy of file is 0.550000
 
iteration 300
cost 0.682617
f1-score is 0.689655
area under curve is 0.550000
accuracy 0.550000
f1-score of file is 0.671533
area under curve of file is 0.550000
accuracy of file is 0.550000
 
iteration 400
cost 0.703904
f1-score is 0.689655
area under curve is 0.550000
accuracy 0.550000
f1-score of file is 0.671533
area under curve of file is 0.550000
accuracy of file is 0.550000
 
iteration 0
cost 0.711477
f1-score is 0.615385
area under curve is 0.500000
accuracy 0.500000
f1-score of file is 0.791667
area under curve of file is 0.494318
accuracy of file is 0.666667
 
iteration 100
cost 0.659468
f1-score is 0.333333
area under curve is 0.600000
accuracy 0.600000
f1-score of file is 0.392857
area under curve of file is 0.593750
accuracy of file is 0.433333
 
iteration 200
cost 0.626251
f1-score 

...

iteration 400
cost 0.673053
f1-score is 0.583333
area under curve is 0.500000
accuracy 0.500000
f1-score of file is 0.897196
area under curve of file is 0.553571
accuracy of file is 0.816667
 
iteration 0
cost 0.614839
f1-score is 0.714286
area under curve is 0.600000
accuracy 0.600000
f1-score of file is 0.900000
area under curve of file is 0.500511
accuracy of file is 0.820000
 
iteration 100
cost 0.625881
f1-score is 0.689655
area under curve is 0.550000
accuracy 0.550000
f1-score of file is 0.900000
area under curve of file is 0.500511
accuracy of file is 0.820000
 
iteration 200
cost 0.673091
f1-score is 0.620690
area under curve is 0.450000
accuracy 0.450000
f1-score of file is 0.900000
area under curve of file is 0.500511
accuracy of file is 0.820000
 
iteration 300
cost 0.729530
f1-score is 0.571429
area under curve is 0.400000
accuracy 0.400000
f1-score of file is 0.900000
area under curve of file is 0.500511
accuracy of file is 0.820000
 
iteration 400
cost 0.679127
f1-score 

iteration 200
cost 0.657218
f1-score is 0.666667
area under curve is 0.500000
accuracy 0.500000
f1-score of file is 0.778947
area under curve of file is 0.496032
accuracy of file is 0.650000
 
iteration 300
cost 0.659973
f1-score is 0.689655
area under curve is 0.550000
accuracy 0.550000
f1-score of file is 0.778947
area under curve of file is 0.496032
accuracy of file is 0.650000
 
iteration 400
cost 0.634689
f1-score is 0.740741
area under curve is 0.650000
accuracy 0.650000
f1-score of file is 0.795699
area under curve of file is 0.551587
accuracy of file is 0.683333
 
iteration 0
cost 0.616214
f1-score is 0.695652
area under curve is 0.650000
accuracy 0.650000
f1-score of file is 0.790698
area under curve of file is 0.626374
accuracy of file is 0.700000
 
iteration 100
cost 0.621242
f1-score is 0.666667
area under curve is 0.600000
accuracy 0.600000
f1-score of file is 0.790698
area under curve of file is 0.626374
accuracy of file is 0.700000
 
iteration 200
cost 0.589922
f1-score 

iteration 0
cost 0.689564
f1-score is 0.571429
area under curve is 0.400000
accuracy 0.400000
f1-score of file is 0.585366
area under curve of file is 0.433333
accuracy of file is 0.433333
 
iteration 100
cost 0.707621
f1-score is 0.166667
area under curve is 0.500000
accuracy 0.500000
f1-score of file is 0.341463
area under curve of file is 0.550000
accuracy of file is 0.550000
 
iteration 200
cost 0.711896
f1-score is 0.000000
area under curve is 0.500000
accuracy 0.500000
f1-score of file is 0.350000
area under curve of file is 0.566667
accuracy of file is 0.566667
 
iteration 300
cost 0.695855
f1-score is 0.461538
area under curve is 0.650000
accuracy 0.650000
f1-score of file is 0.390244
area under curve of file is 0.583333
accuracy of file is 0.583333
 
iteration 400
cost 0.655834
f1-score is 0.666667
area under curve is 0.700000
accuracy 0.700000
f1-score of file is 0.400000
area under curve of file is 0.600000
accuracy of file is 0.600000
 
iteration 0
cost 0.664118
f1-score is

iteration 300
cost 0.666625
f1-score is 0.714286
area under curve is 0.600000
accuracy 0.600000
f1-score of file is 0.685714
area under curve of file is 0.560000
accuracy of file is 0.560000
 
iteration 400
cost 0.656836
f1-score is 0.181818
area under curve is 0.550000
accuracy 0.550000
f1-score of file is 0.169492
area under curve of file is 0.510000
accuracy of file is 0.510000
 
iteration 0
cost 0.713669
f1-score is 0.538462
area under curve is 0.400000
accuracy 0.400000
f1-score of file is 0.717391
area under curve of file is 0.406250
accuracy of file is 0.566667
 
iteration 100
cost 0.652913
f1-score is 0.666667
area under curve is 0.550000
accuracy 0.550000
f1-score of file is 0.717391
area under curve of file is 0.406250
accuracy of file is 0.566667
 
iteration 200
cost 0.653023
f1-score is 0.689655
area under curve is 0.550000
accuracy 0.550000
f1-score of file is 0.717391
area under curve of file is 0.406250
accuracy of file is 0.566667
 
iteration 300
cost 0.703120
f1-score 

area under curve of file is 0.545249
accuracy of file is 0.500000
 
iteration 100
cost 0.735738
f1-score is 0.571429
area under curve is 0.400000
accuracy 0.400000
f1-score of file is 0.619048
area under curve of file is 0.420814
accuracy of file is 0.466667
 
iteration 200
cost 0.676023
f1-score is 0.560000
area under curve is 0.450000
accuracy 0.450000
f1-score of file is 0.651163
area under curve of file is 0.450226
accuracy of file is 0.500000
 
iteration 300
cost 0.745761
f1-score is 0.333333
area under curve is 0.600000
accuracy 0.600000
f1-score of file is 0.425532
area under curve of file is 0.589367
accuracy of file is 0.550000
 


In [ ]:
print ('opening file for write')
label = ["IdFeedBack,Prediction"]
with open('output.csv', 'wb') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=' ')
    spamwriter.writerow(label)

    for file_ in test_file_dir:
        # get features from current indexed file
        features = get_test_data(file_)        
        # training features is np array with max size and padding
        test_features, seq_len = prepare_data(features, max_seq_len)
        # scale features to have 0 mean and 1 variance
        scaled_features = feature_scaling(test_features, batch_size,\
                seq_len)
        # find independent components from given features
        inde_features = independent_components(scaled_features, batch_size,\
                seq_len)
        # evaluate over whole file
        y = np.zeros(0)
        if "Sess05" in file_:
            indices = ["%03d" %(i+1) for i in range(100)]
    #        print file_
            for index in range(0,100,batch_size):
                yfile = observe.eval(feed_dict = {\
                    x:inde_features[index:index+batch_size], \
                    s_len:seq_len[index:index+batch_size]})
                y = np.append(y,yfile)
        else:
            indices = ["%03d" %(i+1) for i in range(60)]
            for index in range(0,60,batch_size):
                yfile = observe.eval(feed_dict = {\
                    x:inde_features[index:index+batch_size], \
                    s_len:seq_len[index:index+batch_size]})
                y = np.append(y,yfile)

        for i in xrange(0,y.shape[0]):
            values = [file_[5:-4]+'_FB'+indices[i]+','+str(int(y[i]))]
            spamwriter.writerow(values)
print ('file write complete')